In [2]:
import re
import time
import numpy as np
from pathlib import Path
from tqdm import tqdm
import pandas as pd 
import numpy as np
import seaborn as sns
import numpy as np
import pandas as pd
from functools import partial
from multiprocessing import Pool
import multiprocessing
import time
import re
import glob
import tqdm
import modin.pandas as pd
# import pandas as pd
# import swifter
# from pandarallel import pandarallel
import ray
ray.init(num_cpus=8)

{'node_ip_address': '210.32.145.179',
 'raylet_ip_address': '210.32.145.179',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-07-31_11-43-36_046910_346224/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-07-31_11-43-36_046910_346224/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-07-31_11-43-36_046910_346224',
 'metrics_export_port': 58816,
 'gcs_address': '210.32.145.179:55436',
 'address': '210.32.145.179:55436',
 'node_id': '94b8df739f5ffd066a88d7af4005a89c46d70368db5958bac2cdb84b'}

In [2]:
top101=pd.read_csv('top72.csv')
drug = top101["drug"].to_list()

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [3]:
drug

['ivermectin',
 'Hydroxychloroquine|HCQ|plaquenil',
 'zinc',
 'vitamin d|Vitamin D2|Vitamin D3|vd3|vd2',
 'Remdesivir|velkury',
 'tylenol|acetaminophen|Paracetamol|APAP',
 'Vitamin C|vit c',
 'fentanyl',
 'Cocaine',
 'aspirin',
 'Chloroquine',
 'ibuprofen|advil|motrin|Nurofen|brufen',
 'Heroin',
 'dexamethasone',
 'Quercetin',
 'azithromycin',
 'viagra|Sildenafil',
 'nicotine',
 'Caffeine',
 'actemra|Tocilizumab|RoActemra',
 'botox',
 'adderall',
 'Tocilizumab',
 'Adrenaline|epipen|Epinephrine|Adrenaclick',
 'penicillin',
 'Molnupiravir',
 'Midazolam',
 'testosterone',
 'magnesium',
 'Thalidomide|Contergan|Thalomid|Talidex',
 'tamiflu|Oseltamivir',
 'prednisone',
 'Fluvoxamine',
 'Morphine',
 'niacin|nicotinic acid',
 'budesonide',
 'doxycycline',
 'Quinine',
 'Interferon',
 'midazolam ',
 'xanax',
 'benadryl|Diphenhydramine',
 'Opium',
 'oxycontin|Roxicodone',
 'morphine ',
 'famotidine|Pepcid',
 'mucinex|Guaifenesin',
 'zithromax',
 'Ketamine',
 'betadine|Povidone-iodine',
 'albutero

In [4]:
df= pd.DataFrame()
for i in drug:
    df_ = pd.DataFrame(index=range(5),columns=[str(i)])
    df_ = df_.fillna(0) # with 0s rather than NaNs
    df=pd.concat([df,df_],axis=1)
print(df)
df.to_csv('top72drame.csv',index=False)

   ivermectin  Hydroxychloroquine|HCQ|plaquenil  zinc  \
0           0                                 0     0   
1           0                                 0     0   
2           0                                 0     0   
3           0                                 0     0   
4           0                                 0     0   

   vitamin d|Vitamin D2|Vitamin D3|vd3|vd2  Remdesivir|velkury  \
0                                        0                   0   
1                                        0                   0   
2                                        0                   0   
3                                        0                   0   
4                                        0                   0   

   tylenol|acetaminophen|Paracetamol|APAP  Vitamin C|vit c  fentanyl  Cocaine  \
0                                       0                0         0        0   
1                                       0                0         0        0   
2                

In [44]:
top101=pd.read_csv('top72drame.csv')
top101

,ivermectin,Hydroxychloroquine|HCQ|plaquenil,zinc,vitamin d|Vitamin D2|Vitamin D3|vd3|vd2,Remdesivir|velkury,tylenol|acetaminophen|Paracetamol|APAP,Vitamin C|vit c,fentanyl,Cocaine,aspirin,...,COVID-19 convalescent plasma,Vaseline,bamlanivimab,codeine,colchicine|Colcrys,ritonavir,yasmin|Drospirenone,vitamin a|Retinol,Methadone|Dolophine,Cortisone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
data_dir='/data1/data_8t/drugtweets/roberta_drug/'
out_dir='/data1/data_8t/lwx/Drugtweets/graphic/drug/'
files = sorted(glob.glob(data_dir+"*.csv"))

In [45]:
#横向连接列名是keys的列，数量就是keys的数量
for file in tqdm.tqdm(files):
    dff = pd.read_csv(file,lineterminator='\n',usecols=['created_at','id','full_text','user','extracted_entity','drug_entity','symp_entity','dis_entity'])
    dff = pd.concat([dff,top101],axis=1,join='outer')
    dff.to_csv(out_dir+file.split("/")[-1],index=False)  #保存文件

100%|██████████| 135/135 [03:34<00:00,  1.59s/it]


In [ ]:
#单个文件填充测试
df = pd.read_csv('/data1/data_8t/lwx/Drugtweets/graphic/21April_0.csv',dtype='string',lineterminator='\n')
for keys in drug:
    df[keys]=df['full_text'].str.contains(fr"\b(?:{keys})\b",case=False,flags=re.I)
df.to_csv('test.csv')

In [47]:
#如果出现了这个keys，就填充true，否则是false 
#既包含连接 又包含填充
for file in tqdm.tqdm(files):
    dff = pd.read_csv(file,lineterminator='\n',usecols=['created_at','id','full_text','user','extracted_entity','drug_entity','symp_entity'])
    dff = pd.concat([dff,top101],axis=1,join='outer')
    for keys in drug:
        dff[keys]=dff['drug_entity'].str.contains(fr"\b(?:{keys})\b",case=False,flags=re.I)#连接并填充
    dff.to_csv(out_dir+file.split("/")[-1],index=False)  #保存文件

100%|██████████| 135/135 [08:45<00:00,  3.89s/it]


In [28]:
out_dir='/data1/data_8t/lwx/Drugtweets/graphic/drug/'
files = sorted(glob.glob('/data1/data_8t/lwx/Drugtweets/graphic/drug/'+"*.csv"))

empty=pd.read_csv('/data1/data_8t/lwx/Drugtweets/graphic/drug/21April_0.csv',dtype='string',lineterminator='\n')
for file in tqdm.tqdm(files[1:]):
    dff = pd.read_csv(file,lineterminator='\n')
    empty=pd.concat([empty,dff],axis=0)
#empty.to_csv('drug_data.csv',index=False)

In [4]:
import numpy as np
from time import sleep
import ray
import psutil
import gc
def auto_garbage_collect(pct=80.0):
    if psutil.virtual_memory().percent >= pct:
        gc.collect()

gc.set_threshold(99)

In [30]:
empty

,created_at,id,full_text,user,extracted_entity,drug_entity,symp_entity,ivermectin,Hydroxychloroquine|HCQ|plaquenil,zinc,...,COVID-19 convalescent plasma,Vaseline,bamlanivimab,codeine,colchicine|Colcrys,ritonavir,yasmin|Drospirenone,vitamin a|Retinol,Methadone|Dolophine,Cortisone
0,Wed Apr 21 02:39:50 +0000 2021,1384698359475097601,@drharshvardhan @PMOIndia \nCan you look in to...,"{'id': 279384290, 'id_str': '279384290', 'name...","[{'type': 'Drug', 'start_index': 11, 'end_inde...",['clo2 gas'],[],False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Fri Apr 16 04:01:18 +0000 2021,1382906924124012544,@gathertonourish @Jo_Brant In Quebec and USA I...,"{'id': 899809505101459456, 'id_str': '89980950...","[{'type': 'Location', 'start_index': 3, 'end_i...",['ivermectin'],[],True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Sat Apr 17 01:16:03 +0000 2021,1383227726102818822,Ayurveda.. #healing.The distinction between he...,"{'id': 1260691841285931008, 'id_str': '1260691...","[{'type': 'Drug', 'start_index': 0, 'end_index...",['ayurveda'],[],False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Sat Apr 17 20:02:07 +0000 2021,1383511111014649868,@PrettyKaur1992 Farmers r protesting frm past ...,"{'id': 1357354600790036482, 'id_str': '1357354...","[{'type': 'Organization', 'start_index': 21, '...",['remdesivir'],[],False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Thu Apr 22 03:02:25 +0000 2021,1385066431281131520,7. Our Health Minister @drharshvardhan ji kept...,"{'id': 1182964773970825217, 'id_str': '1182964...","[{'type': 'Person', 'start_index': 5, 'end_ind...",['ayurveda medicines'],[],False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12487,Sat Sep 19 01:37:38 +0000 2020,1307131705782927361,@MikeBalsamo1 @redsteeze Social distancing a t...,"{'id': 1292771833, 'id_str': '1292771833', 'na...","[{'type': 'Drug', 'start_index': 9, 'end_index...",['rbg'],[],False,False,False,...,False,False,False,False,False,False,False,False,False,False
12488,Mon Sep 28 14:42:04 +0000 2020,1310590603822952449,#TCOT RedHill Biopharma’s Opaganib Demonstrate...,"{'id': 79504323, 'id_str': '79504323', 'name':...","[{'type': 'Organization', 'start_index': 1, 'e...",['opaganib'],[],False,False,False,...,False,False,False,False,False,False,False,False,False,False
12489,Sat Sep 05 17:34:26 +0000 2020,1302299063182745601,It is claimed by researchers that a combinatio...,"{'id': 1252835854512340995, 'id_str': '1252835...","[{'type': 'Drug', 'start_index': 16, 'end_inde...","['vitamins c d', 'zinc', 'quercetin']",[],False,False,True,...,False,False,False,False,False,False,False,False,False,False
12490,Sun Sep 27 15:39:23 +0000 2020,1310242639460159488,I think is going to take more than a pandemic ...,"{'id': 816987847269851137, 'id_str': '81698784...","[{'type': 'Drug', 'start_index': 16, 'end_inde...",['asteroid'],[],False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [31]:
empty.to_csv('drug_data.csv',index=False)

In [33]:
df_drug_filter =empty
df_drug_array = df_drug_filter.drop(columns=["created_at","id","full_text","user",'extracted_entity','drug_entity','symp_entity'])
df_drug_array = df_drug_array.applymap(lambda x: 1 if x==True else 0)
df_drug_array

,ivermectin,Hydroxychloroquine|HCQ|plaquenil,zinc,vitamin d|Vitamin D2|Vitamin D3|vd3|vd2,Remdesivir|velkury,tylenol|acetaminophen|Paracetamol|APAP,Vitamin C|vit c,fentanyl,Cocaine,aspirin,...,COVID-19 convalescent plasma,Vaseline,bamlanivimab,codeine,colchicine|Colcrys,ritonavir,yasmin|Drospirenone,vitamin a|Retinol,Methadone|Dolophine,Cortisone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12487,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12488,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12489,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
drug_data_array = df_drug_array.values
type(drug_data_array), drug_data_array.shape

(numpy.ndarray, (2124757, 70))

In [35]:
data_drug_array_t = drug_data_array.T
type(data_drug_array_t), data_drug_array_t.shape

(numpy.ndarray, (70, 2124757))

In [36]:
dict_drug = {}
for drug_name, drug_code in zip(df_drug_array.columns, data_drug_array_t):
    dict_drug[drug_name] = drug_code
len(dict_drug)

70

In [25]:
a = dict_drug['Hydroxychloroquine|HCQ|plaquenil']
b = dict_drug['ivermectin']
sum(a&b)#两个都是1的时候才会是1 看有多少个两个都是1 就可以统计

38496

In [26]:
a

array([0, 0, 0, ..., 0, 0, 0])

In [37]:
dict_drug

{'ivermectin': array([0, 0, 0, ..., 0, 0, 0]),
 'Hydroxychloroquine|HCQ|plaquenil': array([0, 0, 0, ..., 0, 0, 1]),
 'zinc': array([0, 0, 0, ..., 1, 0, 1]),
 'vitamin d|Vitamin D2|Vitamin D3|vd3|vd2': array([0, 0, 0, ..., 0, 0, 0]),
 'Remdesivir|velkury': array([0, 0, 0, ..., 0, 0, 0]),
 'tylenol|acetaminophen|Paracetamol|APAP': array([0, 0, 0, ..., 0, 0, 0]),
 'Vitamin C|vit c': array([0, 0, 0, ..., 0, 0, 0]),
 'fentanyl': array([0, 0, 0, ..., 0, 0, 0]),
 'Cocaine': array([0, 0, 0, ..., 0, 0, 0]),
 'aspirin': array([0, 0, 0, ..., 0, 0, 0]),
 'Chloroquine': array([0, 0, 0, ..., 0, 0, 0]),
 'ibuprofen|advil|motrin|Nurofen|brufen': array([0, 0, 0, ..., 0, 0, 0]),
 'Heroin': array([0, 0, 0, ..., 0, 0, 0]),
 'dexamethasone': array([0, 0, 0, ..., 0, 0, 0]),
 'Quercetin': array([0, 0, 0, ..., 1, 0, 0]),
 'azithromycin': array([0, 0, 0, ..., 0, 0, 0]),
 'viagra|Sildenafil': array([0, 0, 0, ..., 0, 0, 0]),
 'nicotine': array([0, 0, 0, ..., 0, 0, 0]),
 'Caffeine': array([0, 0, 0, ..., 0, 0, 0])

In [38]:
df_dist_drug = pd.DataFrame(columns=['Source','Target', 'weight'], index=range(20550))
df_dist_drug

,Source,Target,weight
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
20545,NaN,NaN,NaN
20546,NaN,NaN,NaN
20547,NaN,NaN,NaN
20548,NaN,NaN,NaN


In [75]:
total=pd.read_csv('/data_8t/lwx/Drugtweets/graphic/network/ttotal_net.csv')
total

,drug1,drug2,weight
0,acetaminophen,advil,1
1,acetaminophen,aspirin,8
2,acetaminophen,caffeine,1
3,acetaminophen,ibuprofen,22
4,actemra,azithromycin,1
...,...,...,...
38533,vitamin d,vitamin e,3
38534,vitamin d,zinc,139
38535,vitamin d3,zinc,24
38536,vitamin e,zinc,2


In [39]:
df_drug_drug=pd.DataFrame(columns=drug,index=drug)
df_drug_drug

,ivermectin,Hydroxychloroquine|HCQ|plaquenil,zinc,vitamin d|Vitamin D2|Vitamin D3|vd3|vd2,Remdesivir|velkury,tylenol|acetaminophen|Paracetamol|APAP,Vitamin C|vit c,fentanyl,Cocaine,aspirin,...,COVID-19 convalescent plasma,Vaseline,bamlanivimab,codeine,colchicine|Colcrys,ritonavir,yasmin|Drospirenone,vitamin a|Retinol,Methadone|Dolophine,Cortisone
ivermectin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hydroxychloroquine|HCQ|plaquenil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zinc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Remdesivir|velkury,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ritonavir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
yasmin|Drospirenone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vitamin a|Retinol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Methadone|Dolophine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
#threshold_co = 1
idx_drug_drug = 0
for idx_1, name_drug_1 in enumerate(df_drug_drug.columns):
    for idx_2, name_drug_2 in enumerate(df_drug_drug.columns[idx_1+1:]):
        #count_co = df_drug_drug.loc[name_drug_1, name_drug_2]
        #if count_co >= threshold_co:
        vector_1, vector_2 = dict_drug[name_drug_1], dict_drug[name_drug_2]
        dist_pair = np.dot(vector_1, vector_2) / (np.linalg.norm(vector_1)*(np.linalg.norm(vector_2)))
        df_dist_drug.iloc[idx_drug_drug] = [name_drug_1, name_drug_2, dist_pair]
        #print(f"{name_drug_1}-{name_drug_2}: count-{count_co}, dist-{dist_pair}")
        print(f"{name_drug_1}-{name_drug_2}: dist-{dist_pair}")
        idx_drug_drug += 1
df_dist_drug = df_dist_drug.dropna()
df_dist_drug

ivermectin-Hydroxychloroquine|HCQ|plaquenil: dist-0.12812409938834737
ivermectin-zinc: dist-0.07164623399592776
ivermectin-vitamin d|Vitamin D2|Vitamin D3|vd3|vd2: dist-0.029511194203671865
ivermectin-Remdesivir|velkury: dist-0.023194163207948988
ivermectin-tylenol|acetaminophen|Paracetamol|APAP: dist-0.0096789631924861
ivermectin-Vitamin C|vit c: dist-0.028439272484067524
ivermectin-fentanyl: dist-0.0009769624185575097
ivermectin-Cocaine: dist-0.0006706506459378723
ivermectin-aspirin: dist-0.018853656863547594
ivermectin-Chloroquine: dist-0.01172309865448808
ivermectin-ibuprofen|advil|motrin|Nurofen|brufen: dist-0.004558383438595057
ivermectin-Heroin: dist-0.0009280437585830605
ivermectin-dexamethasone: dist-0.013846793351312123
ivermectin-Quercetin: dist-0.03274299875462239
ivermectin-azithromycin: dist-0.027407181732707838
ivermectin-viagra|Sildenafil: dist-0.00867719279441432
ivermectin-nicotine: dist-0.00034908934539571465
ivermectin-Caffeine: dist-0.0002300462344382858
ivermectin

ivermectin-COVID-19 convalescent plasma: dist-nan
ivermectin-Vaseline: dist-0.0004682363306521013
ivermectin-bamlanivimab: dist-0.0017741110753693092
ivermectin-codeine: dist-0.00045048614025849833
ivermectin-colchicine|Colcrys: dist-0.008207777414283603
ivermectin-ritonavir: dist-0.0031137397812805523
ivermectin-yasmin|Drospirenone: dist-0.0
ivermectin-vitamin a|Retinol: dist-0.0018760887295573543
ivermectin-Methadone|Dolophine: dist-0.00011465306869798816
ivermectin-Cortisone: dist-0.0015351377473984183
Hydroxychloroquine|HCQ|plaquenil-zinc: dist-0.1663641879419894
Hydroxychloroquine|HCQ|plaquenil-vitamin d|Vitamin D2|Vitamin D3|vd3|vd2: dist-0.021223356341832346
Hydroxychloroquine|HCQ|plaquenil-Remdesivir|velkury: dist-0.035907259999885324
Hydroxychloroquine|HCQ|plaquenil-tylenol|acetaminophen|Paracetamol|APAP: dist-0.007478749171015794
Hydroxychloroquine|HCQ|plaquenil-Vitamin C|vit c: dist-0.02263918639566057
Hydroxychloroquine|HCQ|plaquenil-fentanyl: dist-0.0005248286336533469
Hyd

Hydroxychloroquine|HCQ|plaquenil-codeine: dist-0.000386175309186959
Hydroxychloroquine|HCQ|plaquenil-colchicine|Colcrys: dist-0.006655718717729217
Hydroxychloroquine|HCQ|plaquenil-ritonavir: dist-0.017549395456854352
Hydroxychloroquine|HCQ|plaquenil-yasmin|Drospirenone: dist-0.0
Hydroxychloroquine|HCQ|plaquenil-vitamin a|Retinol: dist-0.001943314985480111
Hydroxychloroquine|HCQ|plaquenil-Methadone|Dolophine: dist-6.552356117367287e-05
Hydroxychloroquine|HCQ|plaquenil-Cortisone: dist-0.0015257778852772583
zinc-vitamin d|Vitamin D2|Vitamin D3|vd3|vd2: dist-0.19120413567207453
zinc-Remdesivir|velkury: dist-0.01214886723860221
zinc-tylenol|acetaminophen|Paracetamol|APAP: dist-0.016932002004103008
zinc-Vitamin C|vit c: dist-0.3040224186817396
zinc-fentanyl: dist-5.6038913673114686e-05
zinc-Cocaine: dist-0.0001899691894514834
zinc-aspirin: dist-0.03859679476348376
zinc-Chloroquine: dist-0.022849196918732793
zinc-ibuprofen|advil|motrin|Nurofen|brufen: dist-0.008690273311981035
zinc-Heroin: di

zinc-codeine: dist-0.0009690017284193992
zinc-colchicine|Colcrys: dist-0.0044535234217373595
zinc-ritonavir: dist-0.0012317601880902208
zinc-yasmin|Drospirenone: dist-0.0
zinc-vitamin a|Retinol: dist-0.03026619010796975
zinc-Methadone|Dolophine: dist-0.0
zinc-Cortisone: dist-0.0011485571755976622
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-Remdesivir|velkury: dist-0.003531008941089777
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-tylenol|acetaminophen|Paracetamol|APAP: dist-0.006197868602785256
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-Vitamin C|vit c: dist-0.10390857356062276
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-fentanyl: dist-3.7982388732522134e-05
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-Cocaine: dist-0.00014485325497951896
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-aspirin: dist-0.015243443728965612
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-Chloroquine: dist-0.0017669577288567339
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-ibuprofen|advil|motrin|Nurofen|brufen: dist-0.0029192386363923292
vitamin d|Vi

vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-bamlanivimab: dist-0.0020901203021195286
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-codeine: dist-0.00010946262730335866
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-colchicine|Colcrys: dist-0.0016170726612709774
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-ritonavir: dist-0.00010435871973311706
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-yasmin|Drospirenone: dist-0.0
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-vitamin a|Retinol: dist-0.01846259876405069
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-Methadone|Dolophine: dist-0.0
vitamin d|Vitamin D2|Vitamin D3|vd3|vd2-Cortisone: dist-0.000648729937015292
Remdesivir|velkury-tylenol|acetaminophen|Paracetamol|APAP: dist-0.003469510976110328
Remdesivir|velkury-Vitamin C|vit c: dist-0.005211712310045745
Remdesivir|velkury-fentanyl: dist-0.0010595662421110132
Remdesivir|velkury-Cocaine: dist-8.784484196257528e-05
Remdesivir|velkury-aspirin: dist-0.004850936559377906
Remdesivir|velkury-Chloroquine: dist-0.01071554259984819
Remdes

...

Menthol-codeine: dist-0.0016401996283796187
Menthol-colchicine|Colcrys: dist-0.0
Menthol-ritonavir: dist-0.0
Menthol-yasmin|Drospirenone: dist-0.0
Menthol-vitamin a|Retinol: dist-0.0
Menthol-Methadone|Dolophine: dist-0.0
Menthol-Cortisone: dist-0.0
COVID-19 convalescent plasma-Vaseline: dist-nan


COVID-19 convalescent plasma-bamlanivimab: dist-nan
COVID-19 convalescent plasma-codeine: dist-nan
COVID-19 convalescent plasma-colchicine|Colcrys: dist-nan


COVID-19 convalescent plasma-ritonavir: dist-nan
COVID-19 convalescent plasma-yasmin|Drospirenone: dist-nan
COVID-19 convalescent plasma-vitamin a|Retinol: dist-nan
COVID-19 convalescent plasma-Methadone|Dolophine: dist-nan


COVID-19 convalescent plasma-Cortisone: dist-nan
Vaseline-bamlanivimab: dist-0.0
Vaseline-codeine: dist-0.0
Vaseline-colchicine|Colcrys: dist-0.0
Vaseline-ritonavir: dist-0.0
Vaseline-yasmin|Drospirenone: dist-0.0
Vaseline-vitamin a|Retinol: dist-0.0
Vaseline-Methadone|Dolophine: dist-0.0
Vaseline-Cortisone: dist-0.0033939474721049745
bamlanivimab-codeine: dist-0.0
bamlanivimab-colchicine|Colcrys: dist-0.0019426853568039809
bamlanivimab-ritonavir: dist-0.0018805848512798901
bamlanivimab-yasmin|Drospirenone: dist-0.0
bamlanivimab-vitamin a|Retinol: dist-0.0
bamlanivimab-Methadone|Dolophine: dist-0.0
bamlanivimab-Cortisone: dist-0.0
codeine-colchicine|Colcrys: dist-0.0
codeine-ritonavir: dist-0.0
codeine-yasmin|Drospirenone: dist-0.0
codeine-vitamin a|Retinol: dist-0.0
codeine-Methadone|Dolophine: dist-0.00420677103075225
codeine-Cortisone: dist-0.0012244828600917394
colchicine|Colcrys-ritonavir: dist-0.009699736253207457
colchicine|Colcrys-yasmin|Drospirenone: dist-0.0
colchicine|Colcry

,Source,Target,weight
0,ivermectin,Hydroxychloroquine|HCQ|plaquenil,0.128124
1,ivermectin,zinc,0.071646
2,ivermectin,vitamin d|Vitamin D2|Vitamin D3|vd3|vd2,0.029511
3,ivermectin,Remdesivir|velkury,0.023194
4,ivermectin,tylenol|acetaminophen|Paracetamol|APAP,0.009679
...,...,...,...
2410,yasmin|Drospirenone,Methadone|Dolophine,0.0
2411,yasmin|Drospirenone,Cortisone,0.0
2412,vitamin a|Retinol,Methadone|Dolophine,0.0
2413,vitamin a|Retinol,Cortisone,0.0


In [ ]:
count_cos = []
dist_pairs = []
for idx_1, name_drug_1 in enumerate(drug):
    for idx_2, name_drug_2 in enumerate(drug[idx_1+1:]):
        vector_1 = dict_drug[name_drug_1]
        vector_2 = dict_drug[name_drug_2]
        count_cos.append(sum(vector_1&vector_2))
        dist_pair = np.dot(vector_1, vector_2) / (np.linalg.norm(vector_1)*(np.linalg.norm(vector_2)))
        dist_pairs.append(dist_pair)

In [26]:
len(dist_pairs)

NameError: name 'dist_pairs' is not defined

In [41]:
df_dist_drug = df_dist_drug.dropna()
df_dist_drug

,Source,Target,weight
0,ivermectin,Hydroxychloroquine|HCQ|plaquenil,0.128124
1,ivermectin,zinc,0.071646
2,ivermectin,vitamin d|Vitamin D2|Vitamin D3|vd3|vd2,0.029511
3,ivermectin,Remdesivir|velkury,0.023194
4,ivermectin,tylenol|acetaminophen|Paracetamol|APAP,0.009679
...,...,...,...
2410,yasmin|Drospirenone,Methadone|Dolophine,0.0
2411,yasmin|Drospirenone,Cortisone,0.0
2412,vitamin a|Retinol,Methadone|Dolophine,0.0
2413,vitamin a|Retinol,Cortisone,0.0


In [42]:
df_dist_drug.to_excel("dist_drug_70.xlsx", index=None)
df_dist_drug.to_csv("dist_drug_70.csv", index=None)

In [31]:
df_drug_class = pd.read_csv("class.csv")
df_drug_class

,drug,freq,class
0,Hydroxychloroquine|HCQ,648160,抗寄生虫药
1,ivermectin,535287,抗寄生虫药
2,vitamin d|Vitamin D2|Vitamin D3|vd3|vd2|Calcif...,234984,消化系统
3,Remdesivir|velkury,184428,抗感染药
4,zinc,125362,消化系统
...,...,...,...
97,Niacinamide,1058,皮肤
98,Thimerosal,1042,皮肤
99,nitazoxanide,1033,抗寄生虫药
100,Bromhexine,1030,呼吸系统


In [29]:
dict_drug_id = {}
for idx_drug, name_drug in enumerate(df_drug_class['drug']):
    dict_drug_id[name_drug] = idx_drug

NameError: name 'df_drug_class' is not defined

In [33]:
df_dist_drug

,Source,Target,weight
0,Hydroxychloroquine|HCQ,ivermectin,0.089421
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
20545,NaN,NaN,NaN
20546,NaN,NaN,NaN
20547,NaN,NaN,NaN
20548,NaN,NaN,NaN


In [ ]:
df_dist_drug_id = df_dist_drug
df_dist_drug_id['Source'] = df_dist_drug_id['Source'].apply(lambda x: dict_drug_id[x])
df_dist_drug_id['Target'] = df_dist_drug_id['Target'].apply(lambda x: dict_drug_id[x])
df_dist_drug_id

In [28]:
dict_drug_id

{'Hydroxychloroquine|HCQ': 0,
 'ivermectin': 1,
 'vitamin d|Vitamin D2|Vitamin D3|vd3|vd2|Calcifediol|vit d': 2,
 'Remdesivir|velkury': 3,
 'zinc': 4,
 'Vitamin C|vit c': 5,
 'Chloroquine': 6,
 'tylenol|acetaminophen|Paracetamol|APAP': 7,
 'dexamethasone': 8,
 'Quercetin': 9,
 'Cocaine': 10,
 'fentanyl': 11,
 'azithromycin': 12,
 'estradiol': 13,
 'aspirin': 14,
 'ibuprofen|advil|motrin|Nurofen|brufen': 15,
 'Heroin': 16,
 'actemra|Tocilizumab|RoActemra': 17,
 'nicotine': 18,
 'viagra|Sildenafil': 19,
 'Caffeine': 20,
 'botox': 21,
 'Interferon': 22,
 'penicillin': 23,
 'adderall': 24,
 'Thalidomide|Thalomid': 25,
 'testosterone': 26,
 'budesonide': 27,
 'Serotonin*': 28,
 'Adrenaline': 29,
 'niacin': 30,
 'doxycycline': 31,
 'Quinine': 32,
 'magnesium': 33,
 'tamiflu|Oseltamivir': 34,
 'bamlanivimab': 35,
 'Morphine': 36,
 'Fluvoxamine': 37,
 'Midazolam': 38,
 'Opium': 39,
 'prednisone': 40,
 'colchicine': 41,
 'Heparin': 42,
 'xanax': 43,
 'Molnupiravir': 44,
 'famotidine|Pepcid': 45

#### 提取drug实体

In [3]:
data_dir='/data1/data_8t/drugtweets/roberta_drug/'
out_dir='/data1/data_8t/lwx/Drugtweets/graphic/drug/'
files = sorted(glob.glob(data_dir+"*.csv"))

In [4]:
drug=[]
for file in tqdm.tqdm(files):
    dff = pd.read_csv(file,dtype='string',lineterminator='\n')#,usecols=['created_at','id','full_text','user','clean_text','clean_demojized_text','extracted_entity','keywords','drug_entity','drug_entity'])
    s=dff['drug_entity'].tolist()
    drug.extend(s)

  0%|          | 0/135 [00:00<?, ?it/s]UserWarning: `Series.tolist` defaulting to pandas implementation.
Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.
100%|██████████| 135/135 [00:39<00:00,  3.46it/s]


In [ ]:
a=[]
for i in drug:
    b=(i[1:-1])
    if b!='':
       a.append((eval(b)))


In [ ]:
b=[]
for i in a:
    if type(i)==tuple:
        i=list(i)
        for ii in i:
            print(type(ii),ii)
            b.append(ii)
    else:
        print(type(i),i)
        b.append(i)

In [ ]:
dict = {}
for key in b:
    dict[key] = dict.get(key, 0) + 1
#print(dict)

In [ ]:
list = [dict]
df = pd.DataFrame(list).T
df.to_csv('totaldrug.csv',index=True)